In [ ]:
# 這個 Python 3 環境預裝了許多有用的分析庫
# 它由 kaggle/python Docker 映像定義：https://github.com/kaggle/docker-python
# 例如，以下是一些有用的庫載入示例

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 輸入數據文件可在只讀的 "../input/" 目錄中找到
# 例如，運行以下代碼（點擊運行或按 Shift+Enter）將列出輸入目錄下的所有文件

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 您可以向當前目錄（/kaggle/working/）寫入最多 20GB 的數據，這些數據在使用 "Save & Run All" 創建版本時將作為輸出保留 
# 您也可以將臨時文件寫入 /kaggle/temp/，但這些文件在當前會話結束後不會保存

# 關於數據集
這個動物分類數據集包含各種動物的圖像，包括狗、貓、鴨子、鹿、馬、山羊、獅子、老虎、青蛙和鳥類。本專案的目的是使用 TensorFlow 和 Keras 函式庫構建一個深度學習模型，可以準確地將這些圖像分類到它們各自的類別中。通過在數據集上訓練一個卷積神經網絡（CNN），我們旨在實現高準確率，能夠識別給定圖像中的動物。

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# 設定數據集的路徑
data_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset"

# 獲取動物類別的列表
categories = os.listdir(data_path)

# 創建一個列表來存儲圖像文件名和標籤
data = []
labels = []

# 加載圖像和標籤
for category in categories:
    folder_path = os.path.join(data_path, category)
    for image_filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_filename)
        data.append(image_path)
        labels.append(category)

# 將數據拆分為訓練集和測試集
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:

# 定義一個函數來預處理圖像
def preprocess_image(image_path):
    # 加載圖像並將其調整為 (150, 150)
    img = load_img(image_path, target_size=(150, 150))
    # 將圖像轉換為 numpy 陣列
    img_array = img_to_array(img)
    # 將像素值縮放到 [0, 1] 範圍
    img_array /= 255.0
    # 返回預處理後的圖像
    return img_array

# 我們的目標是什麼？

本專案的目標是開發一個深度學習模型，能夠準確地將動物圖像分類到它們各自的類別中。模型基於一個包含各種動物圖像的數據集進行訓練，例如狗、貓、鴨子、鹿、馬、山羊、獅子、老虎、青蛙和鳥類。通過使用卷積神經網絡（CNN）和遷移學習技術，模型能夠從圖像中學習特徵並以高準確度進行預測。該專案具有潛力應用於真實世界的場景，例如野生動物保育、動物追蹤和監控。

# 我們試圖實現什麼？

我們試圖構建一個深度學習模型，能夠準確地將動物圖像分類到它們各自的類別中。通過這樣做，我們可以創建一個對多種應用有用的工具，例如野生動物保育、獸醫醫學和動物研究。憑藉一個經過良好訓練的模型，我們可以快速且準確地識別不同的動物物種，這有助於物種追蹤、種群監測和疾病診斷。總體而言，我們的目標是構建一個可靠且高效的動物分類模型，能夠造福人類和動物。

In [ ]:
# 定義一個字典，將類別字串映射到整數標籤
label_map = {category: i for i, category in enumerate(categories)}

# 加載圖像和標籤
for category in categories:
    folder_path = os.path.join(data_path, category)
    for image_filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_filename)
        data.append(image_path)
        labels.append(label_map[category])

# 將整數標籤轉換為 one-hot 編碼向量
y_train = to_categorical(np.array([label_map[label] for label in train_labels]))
y_test = to_categorical(np.array([label_map[label] for label in test_labels]))


In [ ]:
# 預處理訓練集中的圖像
X_train = np.array([preprocess_image(image_path) for image_path in train_data])

In [ ]:
# 預處理測試集中的圖像
X_test = np.array([preprocess_image(image_path) for image_path in test_data])

使用列表生成式（list comprehension）將 `preprocess_image` 函數應用於 `train_data` 中的每個圖像路徑。此生成式的結果是一個經過預處理的圖像陣列列表。

In [ ]:
# 定義模型架構
model = Sequential([
Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
MaxPooling2D((2,2)),
Conv2D(64, (3,3), activation='relu'),
MaxPooling2D((2,2)),
Conv2D(128, (3,3), activation='relu'),
MaxPooling2D((2,2)),
Conv2D(128, (3,3), activation='relu'),
MaxPooling2D((2,2)),
Flatten(),
Dense(512, activation='relu'),
Dropout(0.5),
Dense(len(categories), activation='softmax')
])

# 模型解釋

此模型是一個用於圖像分類的卷積神經網絡（CNN）。它具有四個卷積層，每個卷積層後面都接有一個最大池化層。第一層具有 32 個濾波器，核大小為 (3,3)，使用的激活函數是 ReLU。模型的輸入形狀為 (150, 150, 3)，對應於數據集中圖像的大小。每個最大池化層將特徵圖的空間維度縮小一半。在最後一個最大池化層之後，特徵圖被展平，並傳遞到兩個全連接層，這些層使用隨機失活正則化以防止過度擬合。輸出層使用 softmax 激活函數，為數據集中的每個動物類別生成概率。

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# 在測試集上評估模型
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

# 從 history 對象中獲取訓練準確率
train_acc = history.history['accuracy'][-1]

# 輸出訓練準確率
print('Training accuracy:', train_acc)

In [ ]:
model_two = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])


# 選擇哪個模型？

選擇使用模型二而不是模型可能有多種原因。一個可能的原因是，模型二具有比模型更多的層，這可能使其更為複雜，並可能更好地捕捉數據中的潛在模式。然而，這也意味著模型二可能需要更多的計算資源進行訓練，如果沒有適當正則化，可能更容易過度擬合。最終，選擇使用哪個模型將取決於具體任務以及模型複雜度與性能之間的權衡。

In [ ]:
# Compile the model
model_two.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_two = model_two.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model_two.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

train_acc = history_two.history['accuracy'][-1]

# Print the training accuracy
print('Training accuracy:', train_acc)

In [ ]:
model_three = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(len(categories), activation='softmax')
])

# 使用分類交叉熵損失函數和 Adam 優化器編譯模型
model_three.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:

# Train the model on the train set with batch size of 32, for 10 epochs
model_three.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model_three.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

# 數據增強 - 改進圖像分類的可能方法

In [ ]:
# Define an ImageDataGenerator for data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
# 預處理訓練集中的圖像
X_train = np.array([preprocess_image(image_path) for image_path in train_data])

# 使用 ImageDataGenerator 創建數據增強的生成流
flow_train = datagen.flow(X_train, y_train, batch_size=32)

# 預處理測試集中的圖像
X_test = np.array([preprocess_image(image_path) for image_path in test_data])

In [ ]:
model_four = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(len(categories), activation='softmax')
])


In [ ]:
model_four.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=1e-4),
    metrics=['accuracy']
)

In [ ]:
# 定義批次大小和訓練時代數
batch_size = 32
epochs = 50

# 使用數據增強在訓練數據上訓練模型
history = model_four.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test)
)

# 在測試數據上評估模型
test_loss, test_acc = model_three.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

# 模型解釋

模型_four 是一個用於圖像分類的卷積神經網絡（CNN）模型。它包含多個卷積層、最大池化層、一個平坦層、隨機失活層以及全連接層。

模型的第一層是一個 2D 卷積層，具有 32 個 3x3 大小的濾波器，並使用 ReLU 激活函數。它接受大小為 150x150 像素、3 個通道（RGB）的輸入圖像。

第二層是具有 2x2 池化大小的最大池化層，可以將前一層輸出的空間維度減半。

第三層是另一個 2D 卷積層，具有 64 個 3x3 大小的濾波器，並使用 ReLU 激活函數，後接另一個最大池化層。

第四層是一個 2D 卷積層，具有 128 個 3x3 大小的濾波器，並使用 ReLU 激活函數，後接另一個最大池化層。

第五層是一個 2D 卷積層，具有 256 個 3x3 大小的濾波器，並使用 ReLU 激活函數，後接另一個最大池化層。

第六層是另一個 2D 卷積層，具有 256 個 3x3 大小的濾波器，並使用 ReLU 激活函數，後接另一個最大池化層。

第七層是一個平坦層，將前一層的輸出展平成一個一維向量。

第八層是一個隨機失活層，在訓練過程中隨機丟棄 50% 的連接，以防止過度擬合。

第九層是一個具有 512 個神經元的全連接層，並使用 ReLU 激活函數。

最後一層是一個全連接層，其神經元數量等於分類的類別數，並使用 softmax 激活函數輸出各類別的概率分佈。

總體而言，這是一個深層 CNN 架構，具有多個卷積層、池化層和全連接層。卷積層中的高濾波器數量使其能夠學習圖像中的更複雜特徵。隨機失活層用於減少過度擬合，而 softmax 輸出層用於生成最終的分類概率。


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()

In [ ]:
model_five = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D((2,2)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(categories), activation='softmax')
])

此模型包含三個卷積層，每個卷積層的濾波器數量逐漸增加，並且每個卷積層後都接有一個最大池化層。最後一個卷積層的輸出被展平並傳遞到兩個全連接層，其中最後一個全連接層的神經元數量等於需要分類的類別數量。最後一層的激活函數是 softmax，用於輸出各類別的概率分佈。

In [ ]:
# 編譯模型
model_five.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
history = model_five.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# 評估模型
test_loss, test_acc = model_five.evaluate(X_test, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


# 總結
該模型訓練了 10 個時代（epochs），訓練準確率為 0.7417，驗證準確率為 0.2167。在測試集上的測試準確率為 0.2167，測試損失為 2.7339。這表明該模型無法很好地泛化到新數據，並且存在高度的過度擬合問題。

In [ ]:

#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Deer/5.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:

#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Bird/5.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:

#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Horse/30.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt


#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Dog/5.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:
# 編譯模型
model_five.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
history = model_five.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test))

# 評估模型
test_loss, test_acc = model_five.evaluate(X_test, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

# 模型總結

該模型訓練了 30 個時代，最後一個時代的訓練準確率達到了 100%，驗證準確率為 25%。在測試數據上評估後的測試準確率為 25%，測試損失為 5.4038。這表明該模型過度擬合於訓練數據，無法很好地泛化到測試數據。

In [ ]:

#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Horse/25.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:

#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Deer/7.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:

#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Tiger/23.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:

#設定要分類的圖像路徑
image_path = "/kaggle/input/animal-classification-dataset-image-recognition/KaggleDataset/Lion/27.jpg"
# 加載圖像並進行預處理
img = load_img(image_path, target_size=(150, 150))
img_array = img_to_array(img)
img_array /= 255.0
img_array = np.expand_dims(img_array, axis=0)

# 加載訓練過的模型
model = model_five

# 使用模型預測圖像的類別
prediction = model.predict(img_array)[0]

# 獲取預測概率最高的類別
predicted_category = categories[np.argmax(prediction)]

# 顯示圖像
plt.imshow(img)
plt.axis('off')
plt.title(predicted_category)
plt.show()

# 打印預測的類別
print("The predicted category is:", predicted_category)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()

# 關鍵學習點：

在本專案中，我們開發了一個圖像分類模型，用於根據圖像對不同的動物進行分類。我們使用了一個包含 10 個不同類別、共 300 張動物圖像的數據集。所有圖像均從網路上的不同來源收集。

我們通過調整圖像大小、將數據分為訓練集和測試集，以及對像素值進行歸一化來預處理數據。隨後，我們使用 Keras 框架構建了一個卷積神經網絡（CNN）來分類圖像。

總的來說，我們成功地開發了一個可以根據圖像準確分類不同動物的模型。模型的準確率還可以通過調整超參數、使用更多數據或嘗試不同的架構來進一步提高。

# Citations

1. Numpy documentation: https://numpy.org/doc/
2. Pandas documentation: https://pandas.pydata.org/docs/
3. Matplotlib documentation: https://matplotlib.org/stable/contents.html
4. Seaborn documentation: https://seaborn.pydata.org/index.html
5. Sklearn documentation: https://scikit-learn.org/stable/
6. Tensorflow documentation: https://www.tensorflow.org/api_docs
7. Keras documentation: https://keras.io/api/
8. OpenCV documentation: https://docs.opencv.org/
9. Pillow documentation: https://pillow.readthedocs.io/en/stable/
10. PyTorch documentation: https://pytorch.org/docs/stable/index.html
11. Python documentation: https://docs.python.org/3/
12. Stack Overflow: https://stackoverflow.com/
13. Medium articles: https://medium.com/
14. Towards Data Science articles: https://towardsdatascience.com/
15. Machine Learning Mastery: https://machinelearningmastery.com/


# **Licences**
Licences:
MIT License

Copyright (c) 2023 AI SkunkWorks, Muskan Srivastava

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.